In [3]:
import pandas as pd
from sqlalchemy import create_engine
import py
from config import gkey
import requests
import json
from pprint import pprint

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Yelp Data

### Yelp Transform

In [4]:
yelp_json = 'Resources/yelp_business.json'

#adding the commas to the json file
with open (yelp_json,'r', encoding='utf-8') as cmnts:
    all_entries = cmnts.readlines()
    
parsable_data = "[" + ','.join(all_entries).replace('\n','') + "]"
yelp_df = pd.read_json(parsable_data, encoding = 'utf-8')
yelp_df.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC
3,211 W Monroe St,None,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Phoenix,None,1,33.449999,-112.076979,Geico Insurance,,85003,8,1.5,AZ
4,2005 Alyth Place SE,{'BusinessAcceptsCreditCards': 'True'},8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,51.035591,-114.027366,Action Engine,,T2H 0N5,4,2.0,AB


In [5]:
# create dataframe with only the columns we want
new_yelp_df = yelp_df[['name','categories','address','city','state','postal_code','stars' ]].copy()
new_yelp_df.head()


,name,categories,address,city,state,postal_code,stars
0,Minhas Micro Brewery,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",1314 44 Avenue NE,Calgary,AB,T2E 6L6,4.0
1,CK'S BBQ & Catering,"Chicken Wings, Burgers, Caterers, Street Vendo...",,Henderson,NV,89002,4.5
2,La Bastringue,"Breakfast & Brunch, Restaurants, French, Sandw...",1335 rue Beaubien E,Montréal,QC,H2G 1K7,4.0
3,Geico Insurance,"Insurance, Financial Services",211 W Monroe St,Phoenix,AZ,85003,1.5
4,Action Engine,"Home & Garden, Nurseries & Gardening, Shopping...",2005 Alyth Place SE,Calgary,AB,T2H 0N5,2.0


In [6]:
# rename dataframe columns
rename_yelp_df = new_yelp_df.rename(columns={"name":"Restaurant_Y", "categories":"Categories_Y", "address":"Address_Y", "city":"City_Y", "state":"State_Y", "postal_code":"Postal_Code_Y", "stars":"Rating_Y"})
rename_yelp_df.head()

,Restaurant_Y,Categories_Y,Address_Y,City_Y,State_Y,Postal_Code_Y,Rating_Y
0,Minhas Micro Brewery,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",1314 44 Avenue NE,Calgary,AB,T2E 6L6,4.0
1,CK'S BBQ & Catering,"Chicken Wings, Burgers, Caterers, Street Vendo...",,Henderson,NV,89002,4.5
2,La Bastringue,"Breakfast & Brunch, Restaurants, French, Sandw...",1335 rue Beaubien E,Montréal,QC,H2G 1K7,4.0
3,Geico Insurance,"Insurance, Financial Services",211 W Monroe St,Phoenix,AZ,85003,1.5
4,Action Engine,"Home & Garden, Nurseries & Gardening, Shopping...",2005 Alyth Place SE,Calgary,AB,T2H 0N5,2.0


In [7]:
# create dataframe for only Pittsburgh
pits_yelp_df = rename_yelp_df.loc[rename_yelp_df['City_Y']=='Pittsburgh']
pits_yelp_df.head()


,Restaurant_Y,Categories_Y,Address_Y,City_Y,State_Y,Postal_Code_Y,Rating_Y
9,Maggie & Stella's Gifts,"Flowers & Gifts, Gift Shops, Shopping",209 Oakland Ave,Pittsburgh,PA,15213,3.5
67,Philip Pelusi,"Hair Salons, Beauty & Spas","531 Clairton Blvd, Unit 7",Pittsburgh,PA,15236,4.5
68,FedEx Office Print & Ship Center,"Professional Services, Local Services, Shippin...",1720 Washington Rd,Pittsburgh,PA,15241,3.5
73,China Palace,"Asian Fusion, Caterers, Fast Food, Chinese, Re...",5440 Walnut St,Pittsburgh,PA,15232,3.0
89,Pizza Bellagio,"Pizza, Restaurants",4635 Centre Ave,Pittsburgh,PA,15213,1.5


In [8]:
# limit businesses to only restaurants
pits_yelp_df = pits_yelp_df.dropna()
pits_yelp_df = pits_yelp_df[pits_yelp_df['Categories_Y'].str.contains('Restaurants')]
pits_yelp_df = pits_yelp_df.reset_index(drop=True)
pits_yelp_df.head()


,Restaurant_Y,Categories_Y,Address_Y,City_Y,State_Y,Postal_Code_Y,Rating_Y
0,China Palace,"Asian Fusion, Caterers, Fast Food, Chinese, Re...",5440 Walnut St,Pittsburgh,PA,15232,3.0
1,Pizza Bellagio,"Pizza, Restaurants",4635 Centre Ave,Pittsburgh,PA,15213,1.5
2,Pittsburgh Poke,"Sushi Bars, Food, Thai, Poke, Japanese, Restau...",500 Liberty Ave,Pittsburgh,PA,15222,4.5
3,Schenley Park Visitors Center,"Cafes, Hotels & Travel, Swimming Pools, Travel...",101 Panther Hollow Rd,Pittsburgh,PA,15213,5.0
4,Crazy Mocha,"Food, Cafes, Coffee & Tea, Restaurants",1836 Centre Ave,Pittsburgh,PA,15219,5.0


In [9]:
# error with loading caused by Restaurant_Y at row 1038
print(pits_yelp_df.loc[[1037]])

     Restaurant_Y                          Categories_Y   Address_Y  \
1037         täkō  Restaurants, American (New), Mexican  214 6th St   

          City_Y State_Y Postal_Code_Y  Rating_Y  
1037  Pittsburgh      PA         15222       4.5  


In [10]:
# remove accenture causing error
pits_yelp_df = pits_yelp_df.replace('täkō', 'tako')

In [11]:
# new value
print(pits_yelp_df.loc[[1037]])

     Restaurant_Y                          Categories_Y   Address_Y  \
1037         tako  Restaurants, American (New), Mexican  214 6th St   

          City_Y State_Y Postal_Code_Y  Rating_Y  
1037  Pittsburgh      PA         15222       4.5  


In [12]:
# error with loading required resetting the index again
pits_yelp_df = pits_yelp_df.reset_index(drop=True)
pits_yelp_df.head()

,Restaurant_Y,Categories_Y,Address_Y,City_Y,State_Y,Postal_Code_Y,Rating_Y
0,China Palace,"Asian Fusion, Caterers, Fast Food, Chinese, Re...",5440 Walnut St,Pittsburgh,PA,15232,3.0
1,Pizza Bellagio,"Pizza, Restaurants",4635 Centre Ave,Pittsburgh,PA,15213,1.5
2,Pittsburgh Poke,"Sushi Bars, Food, Thai, Poke, Japanese, Restau...",500 Liberty Ave,Pittsburgh,PA,15222,4.5
3,Schenley Park Visitors Center,"Cafes, Hotels & Travel, Swimming Pools, Travel...",101 Panther Hollow Rd,Pittsburgh,PA,15213,5.0
4,Crazy Mocha,"Food, Cafes, Coffee & Tea, Restaurants",1836 Centre Ave,Pittsburgh,PA,15219,5.0


### Yelp Load

In [13]:
# establish connection
rds_connection_string = "root:password@localhost/restaurant_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [14]:
#export to Mysql
pits_yelp_df.to_sql('yelp_reviews', con = engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)


# Yellow Pages Data

### Yellow Pages Transform

In [15]:
csv_file = "Resources/yellowpages.csv"

yellowpagesraw_df = pd.read_csv(csv_file)
yellowpagesraw_df.head()

,Uniq Id,Url,Name,Street,Zip Code,City,State,Phone,Email,Website,Categories,Rating
0,85c7c0ebada10ae36b0ffe6e72e92ed5,http://www.yellowpages.com/indianapolis-in/mip...,454,7280 Michigan Rd,46268.0,Indianapolis,IN,(317) 298-9001,NaN,NaN,"Continental Restaurants, Restaurants",1.0
1,e81c3082aafaf4e57883140616c516bd,http://www.yellowpages.com/philadelphia-pa/mip...,10 Arts,10 Avenue Of The Arts # Lobby,19102.0,Philadelphia,PA,(215) 523-8273,phila.10arts@ritzcarlton.com,http://www.10arts.com,"Continental Restaurants, French Restaurants, R...",1.5
2,92442b01e2f29fd1d8eef92ba2f91f09,http://www.yellowpages.com/miami-fl/mip/100-mo...,100 Montaditos,244 Biscayne Blvd,33132.0,Miami,FL,(786) 347-3065,NaN,http://www.100montaditos.com,"Spanish Restaurants, Restaurants, Sandwich Shops",2.0
3,f6d2b4e33a6c29860f34ef0217f33b97,http://www.yellowpages.com/indianapolis-in/mip...,120 West Market Fresh Grill,120 W Market St,46204.0,Indianapolis,IN,(317) 822-5824,NaN,NaN,Restaurants,2.5
4,0a78c2aa7744435ce55edde64c9254b6,http://www.yellowpages.com/philadelphia-pa/mip...,13th St Pizza,209 S 13th St,19107.0,Philadelphia,PA,(215) 546-4453,NaN,NaN,"Pizza, American Restaurants, Italian Restauran...",3.0


In [16]:
# create dataframe for only Pittsburgh
yellowpagespitt_df = yellowpagesraw_df.loc[yellowpagesraw_df["City"]=="Pittsburgh"]
yellowpagespitt_df.head()


,Uniq Id,Url,Name,Street,Zip Code,City,State,Phone,Email,Website,Categories,Rating
25,c0dea87ba09e8f82da541963b09a2b72,http://www.yellowpages.com/pittsburgh-pa/mip/5...,5 Fools,9566 Perry Hwy,15237.0,Pittsburgh,PA,(412) 366-3544,NaN,NaN,Restaurants,4.5
45,82f675b150359cf9b7153ba964a963f2,http://www.yellowpages.com/pittsburgh-pa/mip/a...,A&M Gyros,3536 Saw Mill Run Blvd,15227.0,Pittsburgh,PA,(412) 885-1010,NaN,NaN,"Greek Restaurants, Fast Food Restaurants, Medi...",1.0
162,b4cfe5fd4413d7fee6142bec935e8c80,http://www.yellowpages.com/pittsburgh-pa/mip/a...,Anthony's,950 Ohio River Blvd,15202.0,Pittsburgh,PA,(412) 734-4000,KillenR@astampcoin.com,http://anthonysfla.com,"Caterers, Restaurants",1.0
169,65bf7d01bb7fba7337e6efc13c84c4ff,http://www.yellowpages.com/pittsburgh-pa/mip/a...,Antonio's Pizzeria,758 Brookline Blvd,15226.0,Pittsburgh,PA,(412) 388-1000,NaN,http://antonios-pizza.com,"Pizza, Restaurants",4.5
448,0256d1b01ca051b457ab423a66cf8416,http://www.yellowpages.com/pittsburgh-pa/mip/b...,BLIMPIE,307 Smithfield St,15222.0,Pittsburgh,PA,(412) 281-0099,NaN,http://www.blimpie.com,"Delicatessens, Take Out Restaurants, Sandwich ...",4.5


In [17]:
# create dataframe with only the columns we want
yppitt_df = yellowpagespitt_df[["Name", "Street", "City","State", "Rating"]]
yppitt_df.head()


,Name,Street,City,State,Rating
25,5 Fools,9566 Perry Hwy,Pittsburgh,PA,4.5
45,A&M Gyros,3536 Saw Mill Run Blvd,Pittsburgh,PA,1.0
162,Anthony's,950 Ohio River Blvd,Pittsburgh,PA,1.0
169,Antonio's Pizzeria,758 Brookline Blvd,Pittsburgh,PA,4.5
448,BLIMPIE,307 Smithfield St,Pittsburgh,PA,4.5


In [18]:
# rename columns
YP_df = yppitt_df.rename(columns={"Name":"Restaurant_YP", "Street":"Address_YP", "City":"City_YP","State":"State_YP", "Rating":"Rating_YP"})
YP_df = YP_df.reset_index(drop=True)
YP_df.head()

,Restaurant_YP,Address_YP,City_YP,State_YP,Rating_YP
0,5 Fools,9566 Perry Hwy,Pittsburgh,PA,4.5
1,A&M Gyros,3536 Saw Mill Run Blvd,Pittsburgh,PA,1.0
2,Anthony's,950 Ohio River Blvd,Pittsburgh,PA,1.0
3,Antonio's Pizzeria,758 Brookline Blvd,Pittsburgh,PA,4.5
4,BLIMPIE,307 Smithfield St,Pittsburgh,PA,4.5


### Yellow Pages Load

In [19]:
YP_df.to_sql('yp_reviews', con = engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

# Google Data

### Google Extract

In [21]:
# Google Geocode API 
target_city = "Pittsburgh, Pennsylvania"

# set up a parameters dictionary
params = {"address": target_city, "key": gkey}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

# Run request
response = requests.get(base_url, params=params)

# Convert to JSON
jc_geo = response.json()

# Extract lat/lng
lat = jc_geo["results"][0]["geometry"]["location"]["lat"]
lng = jc_geo["results"][0]["geometry"]["location"]["lng"]

# Print results
print(f"{target_city}: {lat}, {lng}")

Pittsburgh, Pennsylvania: 40.44062479999999, -79.9958864


In [22]:
# Google Places API 

# geocoordinates
target_coordinates = "40.44062479999999, -79.9958864"
target_radius = 1000
target_type = "restaurant"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
pitt_data = response.json()

# Print the json (pretty printed)
print(json.dumps(pitt_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "CtQETAIAANyv3HlZpnXbStRVhsQVCj0FdMilYhzzjHflKYXRJDfT3fdsRRX7W6RJeJBnRwPqNvrNrC0O2xTD8BmL49viD0_UIsRA4KxjR27IIsS9u33xMO3yvSf-oFNYaUmbsdBhXraEMmsu40wuvQxtENTWwiiaC-ldrWzKB6CVskSYRsRKxaWKKY4hQAz7FVJCVbbe7nNIVjMfXgKtUVRQyK-cwXsozByBkMe7UkE3psvSUD6E-QVdjtvS7Nrb5duyK5rWBXFrB4AWzoIY-TQNk35OIgEV8rferrHr7pVaY3APisPmGyGsIO4nx5uDs-1D7BPs3jpqHZzMA1Bj1Uwts4zpOLUNRHbG9N6e11iUXbC75M9K0d98eC0V-ur1vb9uDUFWQjTiUrkBknWFhK_W0Kov3zQ1m-QjdqGlrRGa3eZq9sXRC03Grna9Qka0XyfGWeOioPFIRNytQVmGvy6Ch2PYAZBVioTEctXn4s7S7PnXsQ_QFrVTN2rd8NdrATq-jLkh3vKZ-Jl_NsVOp7SVegpqE7ceEd_mxxY-uPEGn8MO5YIYEbQNM0y9cp0w3PK5aeF4pLsZ8OQXotSxjpCMh78-DVquPlZbQFexOvUxH8pPXKcIBStgXToa0NWL8bbsKRObdGF55K3ZW2i6nhzAt6SqRhtmbuK7QlnK96G7HOedOxNrjEHEgn5RRx5PwxRh2YAekZF3dZGUgeO6xnauZJ0w0Vkvy29ipk4WtJ3NOeKBXiL7BXJXkWG2_VUvdXAlrmIA_KVMnQUkINYw_wr8f4eFdhsSEP3_wzaFSpbFw3fBkVC-ugkaFDVrf-YauLI5BajwseWQlIVsZXdZ",
    "results": [
        {
            "geometry": {
                "location": {
       

In [23]:
# show only the "results"
pitt_list = pitt_data["results"]
pprint(pitt_list)


[{'geometry': {'location': {'lat': 40.441487, 'lng': -80.0016733},
               'viewport': {'northeast': {'lat': 40.4428411802915,
                                          'lng': -80.00050346970849},
                            'southwest': {'lat': 40.4401432197085,
                                          'lng': -80.0032014302915}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
  'id': 'a9615fcd824425be653c469675e41811672bac59',
  'name': 'Hotel Fairmont Pittsburgh',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 2048,
              'html_attributions': ['<a '
                                    'href="https://maps.google.com/maps/contrib/112876902215755872335/photos">Hotel '
                                    'Fairmont Pittsburgh</a>'],
              'photo_reference': 'CmRaAAAAo5XS3TSi_Tx6YygmzBlZFpT53-gA9j44mk-6rJfJOEMjlZE95rtqfi5gmFNku0eN32lxQy9knb1VWf4jxVMaUyfOWpIrf8LDbd7RGmPjoPImRdBQ9WxvoSwBhilrv86fEhDi2abRIwSZGJ1PxZc0LwLIGh

In [24]:
# lists to store JSON data
name = []
address = []
rating = []

# loop to append data to lists
for restaurant in pitt_list:
   name.append(restaurant["name"])
   address.append(restaurant["vicinity"])
   rating.append(restaurant["rating"])

df = pd.DataFrame({"name":name,"vicinity":address,"rating":rating})
df

,name,vicinity,rating
0,Hotel Fairmont Pittsburgh,"510 Market Street, Pittsburgh",4.6
1,The Carlton Restaurant,"500 Grant Street, Pittsburgh",4.6
2,The Capital Grille,"301 Fifth Avenue, Pittsburgh",4.6
3,Steelhead Brasserie and Wine Bar,"112 Washington Place, Pittsburgh",3.5
4,Primanti Bros.,"2 South Market Square, Pittsburgh",4.3
5,Braddock's Rebellion,"107 6th Street, Pittsburgh",4.1
6,Nola on the Square,"24 Market Square, Pittsburgh",4.1
7,Morton's The Steakhouse,"625 Liberty Avenue Suite 180, Pittsburgh",4.5
8,Ruth's Chris Steak House,"PPG Place, Six, Pittsburgh",4.5
9,Sharp Edge Bistro,"922 Penn Avenue, Pittsburgh",4.0


### Google Transform

In [25]:
# rename columns
LG_df = df.rename(columns={"name":"Restaurant_G", "vicinity":"Address_G", "rating":"Rating_G"})
LG_df


,Restaurant_G,Address_G,Rating_G
0,Hotel Fairmont Pittsburgh,"510 Market Street, Pittsburgh",4.6
1,The Carlton Restaurant,"500 Grant Street, Pittsburgh",4.6
2,The Capital Grille,"301 Fifth Avenue, Pittsburgh",4.6
3,Steelhead Brasserie and Wine Bar,"112 Washington Place, Pittsburgh",3.5
4,Primanti Bros.,"2 South Market Square, Pittsburgh",4.3
5,Braddock's Rebellion,"107 6th Street, Pittsburgh",4.1
6,Nola on the Square,"24 Market Square, Pittsburgh",4.1
7,Morton's The Steakhouse,"625 Liberty Avenue Suite 180, Pittsburgh",4.5
8,Ruth's Chris Steak House,"PPG Place, Six, Pittsburgh",4.5
9,Sharp Edge Bistro,"922 Penn Avenue, Pittsburgh",4.0


In [26]:
# new data frame with split value columns 
g_df = LG_df["Address_G"].str.split(", ", n = 2, expand = True) 
  
# making seperate address column from new data frame 
LG_df["Address_G"]= g_df[0] 

# add column into dataframe
# LG_df = LG_df.rename(columns = {'Address_G': 'address'})
LG_df


,Restaurant_G,Address_G,Rating_G
0,Hotel Fairmont Pittsburgh,510 Market Street,4.6
1,The Carlton Restaurant,500 Grant Street,4.6
2,The Capital Grille,301 Fifth Avenue,4.6
3,Steelhead Brasserie and Wine Bar,112 Washington Place,3.5
4,Primanti Bros.,2 South Market Square,4.3
5,Braddock's Rebellion,107 6th Street,4.1
6,Nola on the Square,24 Market Square,4.1
7,Morton's The Steakhouse,625 Liberty Avenue Suite 180,4.5
8,Ruth's Chris Steak House,PPG Place,4.5
9,Sharp Edge Bistro,922 Penn Avenue,4.0


### Google Load

In [27]:
LG_df.to_sql('google_reviews', con = engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)